In [1]:
#Importando Bibliotecas
import pandas as pd
import numpy as np
import pickle
import warnings
import os
warnings.filterwarnings("ignore")

#Visualization
import seaborn as sns
import matplotlib.pyplot as plt


# Modelo
import lightgbm as lgb

from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.model_selection import train_test_split

#Avaliação do Modelo
from sklearn.metrics import classification_report


path = input('Por favor adicione o diretorio de trabalho?')
for dirname, _, filename in os.walk(path, topdown = True):
    for filename in filename:
        if filename == 'test.csv':
                teste = pd.read_csv(os.path.join(dirname,filename))
        else:
            if filename == 'train.csv':
                treino = pd.read_csv(os.path.join(dirname,filename))
            else:
                print("Deu ruim irmão!!")
                
X = treino.drop(['Cover_Type','Id','Unnamed: 0'],axis=1) 
y= treino.Cover_Type
test_ids = teste['Id']
teste.drop(['Id','Unnamed: 0'],axis=1,inplace = True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

Por favor adicione o diretorio de trabalho? C:\github\Kaggle_Challenges\Machine_Learning_Competitions\data


In [2]:
def LightBoost(X,y,n_iter):
    
    random_state=2
    
    learning_rate = [0.1,0.01,0.001]
    boosting_type = ['gbdt', 'goss']
    num_iterations = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)] 
    objective = ['multiclass','multiclassova']
    num_leaves= [int(x) for x in np.linspace(10, 70, num = 7)]
    max_depth= [int(x) for x in np.linspace(10, 110, num = 11)]

    
    param = {'learning_rate': learning_rate,
             'boosting_type': boosting_type,
             'num_iterations':num_iterations,
             'objective':objective,
             'num_leaves':num_leaves,
             'max_depth':max_depth}
    
    clf = lgb.LGBMClassifier(min_data= 50,sub_feature= 0.5, metric = 'multi_logloss')
    clf_rcv= RandomizedSearchCV(estimator = clf, param_distributions = param, 
                                   n_iter = n_iter, cv = 5, verbose=1, random_state=random_state, 
                                   n_jobs = -1, scoring = {'NLL':'neg_log_loss', 'Accuracy':'accuracy'}, 
                                   refit='NLL')
    clf_rcv.fit(X,y)
    return clf_rcv.best_estimator_, clf_rcv.cv_results_, clf_rcv.best_params_ 

In [3]:
lgbm_model, results, params1 = LightBoost(X_train,y_train,10)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  8.8min finished
C:\Users\PC\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [5]:
#Salvando o modelo
print('>Salvando o modelo')
pickle.dump(lgbm_model,open('lgbm_model.sav', 'wb'))
print('>Modelo Salvo')

>Salvando o modelo


In [6]:
# Validação Cruzada 
resultado_gbm = []
kf = RepeatedKFold(n_splits = 2, n_repeats=10, random_state=1)

for linhas_treino_gbm, linhas_valid_gbm in kf.split(X_train):

    X_treino_gbm, X_valid_gbm = X_train.iloc[linhas_treino_gbm], X_train.iloc[linhas_valid_gbm]
    y_treino_gbm, y_valid_gbm = y_train.iloc[linhas_treino_gbm], y_train.iloc[linhas_valid_gbm]
    
    
    lgbm_model.fit(X_treino_gbm,y_treino_gbm)
    p_gbm= lgbm_model.predict(X_valid_gbm)
    acc_gbm = np.mean(y_valid_gbm==p_gbm)
    resultado_gbm.append(acc_gbm)
print(np.mean(resultado_gbm))
print(np.std(resultado_gbm))
print(classification_report(y_valid_gbm,p_gbm))

C:\Users\PC\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\PC\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\PC\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\PC\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\PC\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Fou

0.8303488756613756
0.004219826477386384
              precision    recall  f1-score   support

           1       0.78      0.70      0.74       862
           2       0.70      0.68      0.69       841
           3       0.84      0.76      0.80       905
           4       0.94      0.95      0.95       855
           5       0.85      0.92      0.89       876
           6       0.79      0.86      0.82       828
           7       0.92      0.96      0.94       881

    accuracy                           0.83      6048
   macro avg       0.83      0.83      0.83      6048
weighted avg       0.83      0.83      0.83      6048

